In [6]:
import os
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM
import joblib

def download_and_clean_stock_data(stock):
    """
    Download stock data for the Magnificent Seven, clean it, and save as CSV files.
    """
   
    output_dir = "raw_stock_data"
    os.makedirs(output_dir, exist_ok=True)

    def save_cleaned_data(data, stock, output_path):
        # Reset index to turn the Date index into a column
        data.reset_index(inplace=True)
        # Retain only the desired columns
        data = data[["Date", "Open", "High", "Low", "Close", "Volume"]]
        # Add a Ticker column
        data["Ticker"] = stock
        # Save the data to CSV
        data.to_csv(output_path, index=False)
        # Reload and remove the first row (if needed)
        data = pd.read_csv(output_path)
        data = data.iloc[1:]
        data.to_csv(output_path, index=False)

    for stock in stocks:
        data = yf.download(stock, start="2015-01-01", end="2025-01-01")
        output_path = os.path.join(output_dir, f"{stock}.csv")
        save_cleaned_data(data, stock, output_path)
        # print(f"Saved cleaned data for {stock} at {output_path}")

def preprocess_data(stock):
    """
    Preprocess each stock's data by converting dates, sorting, and adding new features.
    The processed CSV files are saved into a separate folder.
    """
    data_dir = "raw_stock_data"
    output_dir = "processed_stock_data"
    os.makedirs(output_dir, exist_ok=True)

    def preprocess_stock_data(file_path):
        data = pd.read_csv(file_path)
        # Convert Date to datetime and sort
        data["Date"] = pd.to_datetime(data["Date"])
        data.sort_values(by="Date", inplace=True)
        # Calculate additional features
        data["Daily Return"] = data["Close"].pct_change()
        data["5-Day Moving Avg"] = data["Close"].rolling(window=5).mean()
        data["10-Day Volatility"] = data["Close"].rolling(window=10).std()
        # Remove rows with NaN values from rolling calculations
        data.dropna(inplace=True)
        return data

    for file_name in os.listdir(data_dir):
        if file_name.endswith(".csv"):
            stock_file_path = os.path.join(data_dir, file_name)
            stock_name = os.path.splitext(file_name)[0]
            # print(f"Processing data for {stock_name}...")
            processed_data = preprocess_stock_data(stock_file_path)
            output_path = os.path.join(output_dir, f"{stock_name}_processed.csv")
            processed_data.to_csv(output_path, index=False)
            # print(f"Saved processed data for {stock_name} to {output_path}")
    # print("Data preprocessing complete!")

def load_stock_data(file_path):
    """
    Load processed data from a CSV file and prepare features and target.
    The target is the next day's closing price.
    """
    data = pd.read_csv(file_path)
    features = data[["Open", "High", "Low", "Close", "Volume", "5-Day Moving Avg", "10-Day Volatility"]]
    target = data["Close"].shift(-1)
    # Drop the last row (where target is NaN)
    features = features[:-1]
    target = target[:-1]
    return features, target

def train_model(ticker):
    """
    Load the AAPL processed data, split into training and testing sets,
    build and train a neural network with LSTM layers, and save the model.
    """
    processed_data_dir = "processed_stock_data"
    stock_file_path = os.path.join(processed_data_dir, f"{ticker}_processed.csv")
    features, target = load_stock_data(stock_file_path)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        features.values, target.values, test_size=0.2, random_state=42
    )

    # Reshape data for LSTM: (samples, timesteps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mean_squared_error")

    # print("Training the model...")
    model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=1)

    # print("Evaluating the model on the test set...")
    loss = model.evaluate(X_test, y_test)
    # print(f"Test Loss: {loss}")

    # Save the trained model
    joblib.dump(model,f"models/{ticker}_model.joblib")
    # model.save("models/AAPL_model.h5")
    # print("Model saved as 'models/AAPL_model.h5'")

def evaluate_model():
    """
    Evaluate the saved model on a portion of AAPL's data and # print error metrics.
    Also, predict the next few days' closing prices and save these predictions to CSV.
    """
    processed_data_dir = "processed_stock_data"
    model_path = "stock_price_predictor.h5"
    model = load_model(model_path)

    def load_evaluation_data(file_path):
        data = pd.read_csv(file_path)
        features = data[["Open", "High", "Low", "Close", "Volume", "5-Day Moving Avg", "10-Day Volatility"]]
        target = data["Close"].shift(-1)
        features = features[:-1]
        target = target[:-1]
        # Use the last 10 rows for evaluation (e.g., next days' predictions)
        evaluation_features = features[-10:]
        evaluation_target = target[-10:]
        # Remove the evaluation portion from the main dataset
        features = features[:-10]
        target = target[:-10]
        return features, target, evaluation_features, evaluation_target

    stock_file_path = os.path.join(processed_data_dir, "AAPL_processed.csv")
    features, target, eval_features, eval_target = load_evaluation_data(stock_file_path)

    # Reshape data for LSTM input
    X_train = np.reshape(features.values, (features.shape[0], features.shape[1], 1))
    X_eval = np.reshape(eval_features.values, (eval_features.shape[0], eval_features.shape[1], 1))

    # Evaluate on the training portion (without the last few evaluation rows)
    predicted_train = model.predict(X_train)
    mse = mean_squared_error(target, predicted_train)
    mae = mean_absolute_error(target, predicted_train)
    r2 = r2_score(target, predicted_train)
    
    # print("### Model Evaluation on Training Data ###")
    # print(f"Mean Squared Error (MSE): {mse:.4f}")
    # print(f"Mean Absolute Error (MAE): {mae:.4f}")
    # print(f"R-squared (R2): {r2:.4f}")

    # Predict the next days' prices using the evaluation features
    predicted_eval = model.predict(X_eval)
    results = pd.DataFrame({
        "Index": eval_features.index,
        "Actual Price": eval_target.values,
        "Predicted Price": predicted_eval.flatten()
    })

    # print("### Next Days' Predictions ###")
    # print(results)

    # Save predictions to a CSV file
    # results.to_csv("next_month_predictions.csv", index=False)
    # print("Predictions saved as 'next_month_predictions.csv'")

def main(ticker):
    # print("Starting the stock data pipeline...")
    download_and_clean_stock_data(ticker)
    preprocess_data(ticker)
    train_model(ticker)
    # evaluate_model()
    # print("Pipeline complete!")

def train_model(ticker):
    """
    Load the processed data, split into training and testing sets,
    build and train a neural network with LSTM layers, and save the model
    in TensorFlow SavedModel format (compatible with Vertex AI).
    """
    processed_data_dir = "processed_stock_data"
    stock_file_path = os.path.join(processed_data_dir, f"{ticker}_processed.csv")
    features, target = load_stock_data(stock_file_path)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        features.values, target.values, test_size=0.2, random_state=42
    )

    # Reshape data for LSTM: (samples, timesteps, features)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mean_squared_error")

    print(f"Training the model for {ticker}...")
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

    print(f"Evaluating the model for {ticker}...")
    loss = model.evaluate(X_test, y_test)
    print(f"Test Loss for {ticker}: {loss}")

    # ✅ Save in SavedModel format for Vertex AI
    export_path = f"models/{ticker}_saved_model"
    model.export(export_path)
    print(f"Model saved in TensorFlow SavedModel format at: {export_path}")

for ticker in ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]:
    main(ticker)



[*********************100%***********************]  1 of 1 completed
/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246710471.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ticker"] = stock
[*********************100%***********************]  1 of 1 completed
/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246710471.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ticker"] = stock
[*********************100%***********************]  1 of 1 completed
/

Training the model for AAPL...
Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 12112.5576
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 11258.4502
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10537.9521
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 10020.7949
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9801.7920
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9169.5244
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9168.2549
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8331.9395
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8540.2627
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7679.0684
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7837.5820
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7046.1646
Epoch 13/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7087.4253
Epoch 14/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6614

INFO:tensorflow:Assets written to: models/AAPL_saved_model/assets


Saved artifact at 'models/AAPL_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name='keras_tensor_98')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  5482122576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6416673328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6101094944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6354894368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6354884336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6354888384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6354228272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6354225104: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model saved in TensorFlow SavedModel format at: models/AAPL_saved_model


[*********************100%***********************]  1 of 1 completed
/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246710471.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ticker"] = stock
[*********************100%***********************]  1 of 1 completed
/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246710471.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ticker"] = stock
[*********************100%***********************]  1 of 1 completed
/

...

Model saved in TensorFlow SavedModel format at: models/NVDA_saved_model



/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246710471.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ticker"] = stock
[*********************100%***********************]  1 of 1 completed
/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246710471.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Ticker"] = stock
[*********************100%***********************]  1 of 1 completed
/var/folders/s_/kpvlk3bs5v7_vx6qt436qn400000gn/T/ipykernel_54970/3246

Training the model for TSLA...
Epoch 1/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 25299.3613 
Epoch 2/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 23871.3867
Epoch 3/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24364.3594
Epoch 4/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22973.9863
Epoch 5/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20943.6113
Epoch 6/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21445.8496
Epoch 7/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19892.5156
Epoch 8/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19513.2090
Epoch 9/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 19521.6680
Epoch 10/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17975.8477
Epoch 11/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17934.5898
Epoch 12/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18984.8438
Epoch 13/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 17952.6074
Epoch 14/100
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 

INFO:tensorflow:Assets written to: models/TSLA_saved_model/assets


Saved artifact at 'models/TSLA_saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name='keras_tensor_140')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  13298579792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13299178960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13180892816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13365686720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13365686016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13365676336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13365678272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13365688480: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model saved in TensorFlow SavedModel format at: models/TSLA_saved_model


## Move predict to live

In [2]:
stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]

for stock in stocks:
    predicting_data = pd.read_csv(f"processed_stock_data/{stock}_processed.csv")
    predicting_data = predicting_data.tail(20)
    predicting_data.to_csv(f"/Users/noel_personal/Repos/TIKR/my-react-app/backend/predicting_data/{stock}.csv",index=False)

## Move models to live

In [3]:
stocks = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA"]

for ticker in stocks:
    model = joblib.load(f"models/{stock}_model.joblib")
    joblib.dump(model, f"/Users/noel_personal/Repos/TIKR/my-react-app/backend/live_models/{ticker}_model.joblib")
